<h1>Sliding Window</h1>

### Concept:
Move the left and right boundaries within a certain range without stepping back, to solve problems related to **subarrays** or **substrings**.    
Grow window by r++, and shrink window by l++

### Key to Sliding Window:
Find the **range** and the **answer indicator**, and determine the **monotonic relationship** between them (similar to a greedy approach).

---
<h2>Q1: Minimum Size Subarray Sum (LC.209) --- Sliding Window Template</h2>

*Given an array of positive integers nums and a positive integer target, return the minimal length of a 
subarra 
 whose sum is greater than or equal to target. If there is no such subarray, return 0 instea*d.

**Monotonic Relationship**: Because all elements are positive, larger window size means larger subarray sum

In [11]:
class Solution(object):
    def minSubArrayLen(self, target, nums):
        l, r = 0, 0
        sum, ans = 0, float('inf')
        while r < len(nums):
            sum += nums[r]
            r += 1
            while sum >= target:
                sum -= nums[l]
                l += 1
                ans = min(ans, r - l + 1)
        return ans if ans != float('inf') else 0

---
<h2>Q2: Longest Substring Without Repeating Characters (LC.3)</h2>

*Given a string s, find the length of the longest 
substrin 
 without repeating character*s.

**Monotonic Relationship**: The longer the substring, more repeating characters is likely to appear

In [24]:
class Solution(object):
    def lengthOfLongestSubstring(self, s):
        if not s:
            return 0

        char_set = set()
        l, r, ans = 0, 0, 0

        while r < len(s):
            cur = s[r]
            r += 1
            while cur in char_set:
                char_set.remove(s[l])
                l += 1
            char_set.add(cur)
            size = r - l
            ans = max(ans, size)

        return ans

---
<h2>Q3: Minimum Window Substring (LC.76)</h2>

*Given two strings s and t of lengths m and n respectively, return the minimum window substring of s such that every character in t(including duplicates) is included in the window. If there is no such subtring, return the empty string.*    
*The testcases will be generated such that the answer is unique*

**Monotonic Relationship**: The longer the substring, more characters in t are likely to be covered

In [1]:
class Solution:
    def minWindow(self, s: str, t: str) -> str:
        if len(s) < len(t):
            return ""

        charCnt = defaultdict(int)
        for ch in t:
            charCnt[ch] -= 1
        collected, needed = 0, len(t)
        start, minLen = 0, float('inf')
        l = 0
        for r in range(len(s)):
            if charCnt[s[r]] < 0:
                collected += 1
            charCnt[s[r]] += 1

            if collected == needed:
                # shrink from left while making sure the window is valid
                while charCnt[s[l]] > 0:
                    charCnt[s[l]] -= 1
                    l += 1
                if r - l + 1 < minLen:
                    start = l
                    minLen = r - l + 1
        
        return "" if minLen == float('inf') else s[start: start + minLen]

---
<h2>Q4: Gas Station (LC.134)</h2>

*There are n gas stations along a circular route, where the amount of gas at the ith station is gas[i].*      
*You have a car with an unlimited gas tank and it costs cost[i] of gas to travel from the ith station to its next (i + 1)th station. You begin the journey with an empty tank at one of the gas stations.*         
*Given two integer arrays gas and cost, return the starting gas station's index if you can travel around the circuit once in the clockwise direction, otherwise return -1. If there exists a solution, it is guaranteed to be unique.*

In [52]:
class Solution(object):
    def canCompleteCircuit(self, gas, cost):
        station_visited, tank_level = 0, 0
        num_station = len(gas)
        # l: our starting gas station, we can start at any
        for l in range(num_station):
            while(tank_level >= 0):
                if station_visited == num_station:
                    return l
                # r: the next station we will visit
                r = (l + station_visited) % num_station # handle circular array
                station_visited += 1
                tank_level += gas[r] - cost[r]
            
            # we are outside the while loop because we don't have enough gas, so this starting station has failed
            # discard info about the starting station
            station_visited -= 1
            tank_level -= gas[l] - cost[l]

        return -1 # all starting station failed

---
<h2>Q5: Replace Substring For Balanced String (LC.1234)</h2>

*You are given a string s of length n containing only four kinds of characters: 'Q', 'W', 'E', and 'R'.*    
*A string is said to be balanced if each of its characters appears n / 4 times where n is the length of the string.*     
*Return the minimum length of the substring that can be replaced with any other string of the same length to make s balanced. If s is already balanced, return 0.*

**Solution:**          
Suppose the length is 12, meaning each letter should appear for 3 times          
If the string is not balanced, then there must be some letter whose occurrence > 3, suppose we have 5Q, 1W, 4E, 2R          
we simply need to find the smallest substring that contains the excess letters(2Q and 1E)           
So now our question is just the same as Q3 above

In [ ]:
class Solution:
    def balancedString(self, s):
        n = len(s)
        targetCnt = n // 4
        cnt = {}

        for char in s:
            cnt[char] = cnt.get(char, 0) + 1

        excess = {}
        for char in "QWER":
            if cnt.get(char, 0) > targetCnt:
                excess[char] = cnt[char] - targetCnt

        if not excess:
            return 0

        l = 0
        ans = n
        windowCnt = {}

        for r in range(n):
            # Grow window
            windowCnt[s[r]] = windowCnt.get(s[r], 0) + 1

            while all(windowCnt.get(char, 0) >= excess[char] for char in excess):
                ans = min(ans, r - l + 1)
                # Shrink Window
                windowCnt[s[l]] -= 1
                l += 1

        return ans

---
<h2>Q6: Subarray with K Different Integers (LC.992)</h2>

*Given an integer array nums and an integer k, return the number of good subarrays of nums.*             
*A good array is an array where the number of different integers in that array is exactly k*

**Solution**:               
the key is to realize that:           
number of subarray with exactly K integer = (number of subarr with K or less integer) - (number of subarr with K-1 or less integer)

In [74]:
class Solution:
    def subarraysWithKDistinct(self, nums, k):
        return self.subarraysWithKOrLess(nums, k) - self.subarraysWithKOrLess(nums, k - 1)

    def subarraysWithKOrLess(self, nums, k):
        cnt = {}
        numSubarr = 0
        l = 0
        categoriesCollected = 0

        for r in range(len(nums)):
            # Grow window
            cnt[nums[r]] = cnt.get(nums[r], 0) + 1
            if cnt[nums[r]] == 1:
                categoriesCollected += 1

            # Whenever categoriesCollected is larger than k, shrink window
            while categoriesCollected > k:
                cnt[nums[l]] -= 1
                if cnt[nums[l]] == 0:
                    categoriesCollected -= 1
                l += 1

            # Add the number of valid subarrays ending at r
            numSubarr += r - l + 1

        return numSubarr

---
<h2>Q7: Longest Substring With At Least K Repeating Characters (LC.395)</h2>

*Given a string s and an integer k, return the length of the longest substring of s such that the frequency of each character in this substring is greater than or equal to k.*            
*if no such substring exists, return 0.*

**Solution:**
Divide and conqure:  
because the question mentioned the characters are all just english letters, so here's what we can do:     
>if there is only 1 distinct characters in the substr, what's the longest substr that satisfy the condition?            
>if there are only 2 distinct characters in the substr, what's the longest substr that satisfy the condition?               
if there are only 3 distinct characters in the substr, what's the longest substr that satisfy the condition?              
...               
>if there are 26 distinct characters in the substr, what's the longest substr that satisfy the condition?
> 
Get the result for each case and find the max among them

In [85]:
class Solution:
    def longestSubstring(self, s, k):
        ans = 0
        
        # Try with different numbers of distinct characters in the substring
        for requiredTypes in range(1, 27):
            cnts = [0] * 26
            l = 0
            numTypeCollected = 0
            numTypeSatisfied = 0

            # Expand the right end of the window
            for r in range(len(s)):
                # Update the character count
                idx = ord(s[r]) - ord('a')
                cnts[idx] += 1

                # Update distinct character counters
                if cnts[idx] == 1:
                    numTypeCollected += 1
                if cnts[idx] == k:
                    numTypeSatisfied += 1

                # Shrink the window from the left if there are more types than required
                while numTypeCollected > requiredTypes:
                    idx_left = ord(s[l]) - ord('a')
                    if cnts[idx_left] == 1:
                        numTypeCollected -= 1
                    if cnts[idx_left] == k:
                        numTypeSatisfied -= 1
                    cnts[idx_left] -= 1
                    l += 1

                # Update the answer if all required types have at least k occurrences
                if numTypeSatisfied == requiredTypes:
                    ans = max(ans, r - l + 1)

        return ans

# Fixed Size Sliding Window
Note that other than dynamically grow or shrink window, there are also questions that requires us to use a fixed size window and simply slide!


---
### Q1. Minimum Swaps To Group All 1's Together
*Given a binary array `nums`, return the minimum number of swaps required to group all 1’s present in the array together in any place in the array.*    

**Solution:**      

First count the number of ones in the entire array, `numOnes`.           
Use a sliding window of fixed size --- `numOnes`.          
For each window count the number of zeros in it, as it is  number of times we need to swap for this window.         
Return the min zero count among all windows.        

In [10]:
class Solution:
    def minSwaps(self, nums):
        n = len(nums)
        numOnes = sum(1 for num in nums if num == 1)

        if n == numOnes:
            return 0

        zeroCnt = 0
        l, r = 0, 0
        while r < numOnes:
            if nums[r] == 0:
                zeroCnt += 1
            r += 1

        ans = zeroCnt

        while r < n:
            if nums[r] == 0:
                zeroCnt += 1
            if nums[l] == 0:
                zeroCnt -= 1
            l += 1
            r += 1
            ans = min(ans, zeroCnt)

        return ans